SUHIT DATTA 50249271

SOURAV RANU 50246451

LAB 3 CSE 587

In [3]:
import os
import pyspark
import findspark

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [5]:
# Directory which consists the data 
dir = "DIC_Project_Data"

In [6]:
#creates a subdirectory list 
subDirNameList =[]
for root, dirs, files in os.walk(dir, topdown=False):
    for name in dirs:
        subDirNameList.append(os.path.join(root, name))

#creates a list of dictionary elements of each category type         
listRDD = []
dictRDDElementsAsMap ={}
for eachFolder in subDirNameList:
    folderName = os.path.basename(eachFolder)
    rdd = sc.wholeTextFiles(eachFolder)
    listRDD.append(rdd)
    dictElement = rdd.collectAsMap()
    dictRDDElementsAsMap[folderName] = dictElement

In [7]:
# method to check if a string is blank or not 
def isNotBlank (myString):
    if myString and myString.strip():
        #myString is not None AND myString is not empty or blank
        return True
    #myString is None OR myString is empty or blank
    return False

In [8]:
"""
this creates a list. Each element of a list is basically a dictionary : 
    key = category and
    value = text content  
"""
listAll = []
for name,v1 in dictRDDElementsAsMap.items():
    for key,value in v1.items():
        dataDic = {}
        if isNotBlank(value):
            dataDic['category'] = name
            dataDic['text'] = value 
            listAll.append(dataDic)
 

In [9]:
# this creates the dataframe from the list 
FULLdf = spark.createDataFrame(listAll)

C:\Users\suhit\Anaconda3\lib\site-packages\pyspark\sql\session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [10]:
#splitting into training and test set 
training, test = FULLdf.randomSplit([0.8, 0.2], seed=7)
training.cache()


DataFrame[category: string, text: string]

Creation of Pipeline

In [13]:
# applying Random Forest  

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer , IDF , StringIndexer ,StopWordsRemover
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Configure an ML pipeline
indexer = StringIndexer(inputCol="category", outputCol="label")
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopwordsRemover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")
hashingTF = HashingTF(inputCol=stopwordsRemover.getOutputCol(), outputCol="rawFeatures")
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=15,  maxDepth=12)
pipeline = Pipeline(stages=[indexer, tokenizer,stopwordsRemover, hashingTF,idf, rf])


paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=4)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModelRF = crossval.fit(training)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
predictions = cvModelRF.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))



+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------+
|category|                text|label|               words|            filtered|         rawFeatures|            features|     rawPrediction|         probability|prediction|
+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------+
|Business| Good Tuesday. He...|  2.0|[, good, tuesday....|[, good, tuesday....|(1000,[0,1,2,3,4,...|(1000,[0,1,2,3,4,...| [2.0,5.0,6.0,2.0]|[0.13333333333333...|       2.0|
|Business|Brady Hill used h...|  2.0|[brady, hill, use...|[brady, hill, use...|(1000,[0,1,2,3,8,...|(1000,[0,1,2,3,8,...| [4.0,4.0,4.0,3.0]|[0.26666666666666...|       0.0|
|Business|Good Wednesday. H...|  2.0|[good, wednesday....|[good, wednesday....|(1000,[0,1,2,3,4,...|(1000,[0,1,2,3,4,...| [6.0,5.0,4.0,

In [ ]:
converter = IndexToString(inputCol="categoryIndex", outputCol="originalCategory")
converted = converter.transform(indexed)

In [14]:
# applying Naive Bayes  

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer , IDF , StringIndexer ,StopWordsRemover
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Configure an ML pipeline
indexer = StringIndexer(inputCol="category", outputCol="label")
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopwordsRemover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")
hashingTF = HashingTF(inputCol=stopwordsRemover.getOutputCol(), outputCol="rawFeatures")
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
pipeline = Pipeline(stages=[indexer, tokenizer,stopwordsRemover, hashingTF,idf, nb])


paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=4)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModelNB = crossval.fit(training)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
predictions = cvModelNB.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))



+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|category|                text|label|               words|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Business| Good Tuesday. He...|  2.0|[, good, tuesday....|[, good, tuesday....|(1000,[0,1,2,3,4,...|(1000,[0,1,2,3,4,...|[-7506.8131448156...|[1.61820182577732...|       2.0|
|Business|Brady Hill used h...|  2.0|[brady, hill, use...|[brady, hill, use...|(1000,[0,1,2,3,8,...|(1000,[0,1,2,3,8,...|[-4785.4573792874...|[2.64678005613299...|       1.0|
|Business|Good Wednesday. H...|  2.0|[good, wednesday....|[good, wednesday....|(1000,[0,1,2,3,4,...|(1000,[0,1,2,3,4,...|[-10

In [16]:
# applying Logistic Regression  

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer , IDF , StringIndexer ,StopWordsRemover
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression

# Configure an ML pipeline
indexer = StringIndexer(inputCol="category", outputCol="label")
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopwordsRemover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")
hashingTF = HashingTF(inputCol=stopwordsRemover.getOutputCol(), outputCol="rawFeatures")
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline = Pipeline(stages=[indexer, tokenizer,stopwordsRemover, hashingTF,idf, lr])


paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=4)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModelLR = crossval.fit(training)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
predictions = cvModelLR.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))



+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|category|                text|label|               words|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Business| Good Tuesday. He...|  2.0|[, good, tuesday....|[, good, tuesday....|(1000,[0,1,2,3,4,...|(1000,[0,1,2,3,4,...|[-0.7406603218313...|[0.00401937982547...|       2.0|
|Business|Brady Hill used h...|  2.0|[brady, hill, use...|[brady, hill, use...|(1000,[0,1,2,3,8,...|(1000,[0,1,2,3,8,...|[-0.1155582931381...|[0.08386980169683...|       2.0|
|Business|Good Wednesday. H...|  2.0|[good, wednesday....|[good, wednesday....|(1000,[0,1,2,3,4,...|(1000,[0,1,2,3,4,...|[0.5

RUNNING THE MODELS GENERATED ON A TEST DATA WHICH HAS BEEN SEPARATELY OBTAINED 

In [17]:
# Directory which consists the test data 
test_dir = "Testing_Data"

In [18]:
#creates a subdirectory list 
subDirNameList =[]
for root, dirs, files in os.walk(dir, topdown=False):
    for name in dirs:
        subDirNameList.append(os.path.join(root, name))

#creates a list of dictionary elements of each category type         
listRDD = []
dictRDDElementsAsMap ={}
for eachFolder in subDirNameList:
    folderName = os.path.basename(eachFolder)
    rdd = sc.wholeTextFiles(eachFolder)
    listRDD.append(rdd)
    dictElement = rdd.collectAsMap()
    dictRDDElementsAsMap[folderName] = dictElement

In [19]:
"""
this creates a test list. Each element of a list is basically a dictionary : 
    key = category and
    value = text content  
"""
testlistAll = []
for name,v1 in dictRDDElementsAsMap.items():
    for key,value in v1.items():
        dataDic = {}
        if isNotBlank(value):
            dataDic['category'] = name
            dataDic['text'] = value 
            testlistAll.append(dataDic)
 

In [20]:
# this creates the test dataframe from the testlistAll 
testDf = spark.createDataFrame(testlistAll)

C:\Users\suhit\Anaconda3\lib\site-packages\pyspark\sql\session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [24]:
# Running Random Forest on this new Test Data 

predictions = cvModelRF.transform(testDf)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy for Random Forest on new test data = " + str(accuracy))



+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------+
|category|                text|label|               words|            filtered|         rawFeatures|            features|     rawPrediction|         probability|prediction|
+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------+
|Business|One of the few re...|  2.0|[one, of, the, fe...|[one, remaining, ...|(1000,[0,2,3,9,11...|(1000,[0,2,3,9,11...| [1.0,1.0,9.0,4.0]|[0.06666666666666...|       2.0|
|Business|Deutsche Bank sai...|  2.0|[deutsche, bank, ...|[deutsche, bank, ...|(1000,[2,8,12,20,...|(1000,[2,8,12,20,...|[0.0,2.0,12.0,1.0]|[0.0,0.1333333333...|       2.0|
|Business|Spotify is a hit....|  2.0|[spotify, is, a, ...|[spotify, hit.on,...|(1000,[0,5,10,16,...|(1000,[0,5,10,16,...|[0.0,1.0,13.0,

In [25]:
# Running Naive Bayes on this new Test Data 

predictions = cvModelNB.transform(testDf)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy for Naive Bayes on new test data = " + str(accuracy))

+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|category|                text|label|               words|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Business|One of the few re...|  2.0|[one, of, the, fe...|[one, remaining, ...|(1000,[0,2,3,9,11...|(1000,[0,2,3,9,11...|[-3011.9941663186...|[5.26919189748328...|       2.0|
|Business|Deutsche Bank sai...|  2.0|[deutsche, bank, ...|[deutsche, bank, ...|(1000,[2,8,12,20,...|(1000,[2,8,12,20,...|[-2106.1071354629...|[3.46778455863961...|       2.0|
|Business|Spotify is a hit....|  2.0|[spotify, is, a, ...|[spotify, hit.on,...|(1000,[0,5,10,16,...|(1000,[0,5,10,16,...|[-25

In [26]:
# Running LR on this new Test Data 

predictions = cvModelLR.transform(testDf)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy for Logistic Regression on new test data = " + str(accuracy))

+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|category|                text|label|               words|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Business|One of the few re...|  2.0|[one, of, the, fe...|[one, remaining, ...|(1000,[0,2,3,9,11...|(1000,[0,2,3,9,11...|[-0.3172092509106...|[0.04569309638105...|       2.0|
|Business|Deutsche Bank sai...|  2.0|[deutsche, bank, ...|[deutsche, bank, ...|(1000,[2,8,12,20,...|(1000,[2,8,12,20,...|[-0.4055738077416...|[0.06206982346279...|       2.0|
|Business|Spotify is a hit....|  2.0|[spotify, is, a, ...|[spotify, hit.on,...|(1000,[0,5,10,16,...|(1000,[0,5,10,16,...|[-0.